# Import Moduals

In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout

from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.initializers import GlorotUniform
from keras.metrics import CategoricalAccuracy
from keras.losses import MeanSquaredError

from NeuralNetwork import MyModel, MyLayer, InputLayer

metrics = CategoricalAccuracy()

# Preprocessing

In [2]:
# import dataset
from keras.datasets import mnist
# load dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# compute the number of labels
num_labels = len(np.unique(y_train))
# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size
input_size
# resize and normalize
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255
# network parameters
batch_size = 128
hidden_units = 256

# Keras' Model

In [4]:
model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size, use_bias=False))
model.add(Dropout(0.45))
model.add(Dense(hidden_units, use_bias=False))
model.add(Dropout(0.45))
model.add(Dense(num_labels, use_bias=False))
model.compile(loss=MeanSquaredError(), 
              optimizer=SGD(learning_rate=0.01),
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=batch_size)

Epoch 1/20
469/469 [==============================] - 1s 2ms/step - loss: 0.3759 - accuracy: 0.2471
Epoch 2/20
469/469 [==============================] - 1s 2ms/step - loss: 0.1966 - accuracy: 0.3556
Epoch 3/20
469/469 [==============================] - 1s 2ms/step - loss: 0.1482 - accuracy: 0.4219
Epoch 4/20
469/469 [==============================] - 1s 2ms/step - loss: 0.1239 - accuracy: 0.4709
Epoch 5/20
469/469 [==============================] - 1s 2ms/step - loss: 0.1084 - accuracy: 0.5084
Epoch 6/20
469/469 [==============================] - 1s 2ms/step - loss: 0.0983 - accuracy: 0.5427
Epoch 7/20
469/469 [==============================] - 1s 2ms/step - loss: 0.0906 - accuracy: 0.5690
Epoch 8/20
469/469 [==============================] - 1s 2ms/step - loss: 0.0852 - accuracy: 0.5921
Epoch 9/20
469/469 [==============================] - 1s 2ms/step - loss: 0.0806 - accuracy: 0.6156
Epoch 10/20
469/469 [==============================] - 1s 2ms/step - loss: 0.0772 - accuracy: 0.6307

In [4]:
y_hat = model.predict(x_test)
metrics.update_state(y_test, y_hat)
metrics.result().numpy()

313/313 [==============================] - 0s 763us/step


0.0881

# My Model

In [3]:
initializer = GlorotUniform()

m = MyModel()
m.addLayer(InputLayer(input_size))
m.addLayer(MyLayer(input_size, hidden_units).activeNode(0.45))
m.addLayer(MyLayer(hidden_units, hidden_units).activeNode(0.45))
m.addLayer(MyLayer(hidden_units, num_labels).activeNode(0.45))

#m.selectNodes([list(range(1,10)), list(range(10, 100))])

m.fit(x_train, y_train, 0.01, epoches = 20, batch_size = batch_size)

469/469: 0.125: 100%|||||||||||||||||||||||||||||||
469/469: 0.115: 100%|||||||||||||||||||||||||||||||
469/469: 0.219: 100%|||||||||||||||||||||||||||||||
469/469: 0.208: 100%|||||||||||||||||||||||||||||||
469/469: 0.250: 100%|||||||||||||||||||||||||||||||
469/469: 0.292: 100%|||||||||||||||||||||||||||||||
469/469: 0.177: 100%|||||||||||||||||||||||||||||||
469/469: 0.312: 100%|||||||||||||||||||||||||||||||
469/469: 0.229: 100%|||||||||||||||||||||||||||||||
469/469: 0.344: 100%|||||||||||||||||||||||||||||||
469/469: 0.292: 100%|||||||||||||||||||||||||||||||
469/469: 0.292: 100%|||||||||||||||||||||||||||||||
469/469: 0.385: 100%|||||||||||||||||||||||||||||||
469/469: 0.240: 100%|||||||||||||||||||||||||||||||
469/469: 0.312: 100%|||||||||||||||||||||||||||||||
469/469: 0.312: 100%|||||||||||||||||||||||||||||||
469/469: 0.344: 100%|||||||||||||||||||||||||||||||
469/469: 0.271: 100%|||||||||||||||||||||||||||||||
469/469: 0.396: 100%|||||||||||||||||||||||||||||||
469/469: 0.4

In [4]:
y_hat = m.predict(x_test)
metrics.reset_state()
metrics.update_state(y_test, y_hat)
metrics.result().numpy()

0.3639